# Design Day Sizing Workflow

This notebook demonstrates the complete workflow from finding a weather
station by address to applying ASHRAE design day conditions and running
a sizing simulation.

## Setup

Install idfkit with weather support:

```bash
pip install idfkit[weather]
```

In [ ]:
from idfkit.weather import (
    DesignDayManager,
    StationIndex,
    WeatherDownloader,
    geocode,
)

## Step 1: Geocode the Project Address

Convert the project address to latitude/longitude coordinates.

In [ ]:
project_address = "Willis Tower, Chicago, IL"

lat, lon = geocode(project_address)
print(f"Project location: {lat:.4f}, {lon:.4f}")

## Step 2: Find the Nearest Weather Station

Search the station index for the nearest weather data.

In [ ]:
index = StationIndex.load()

results = index.nearest(lat, lon, limit=5)

print("Nearest weather stations:")
for r in results:
    print(f"  {r.station.display_name}: {r.distance_km:.1f} km")

# Use the nearest station
station = results[0].station
print(f"\nSelected: {station.display_name}")

## Step 3: Download Weather Files

Download the EPW and DDY files for this station.

In [ ]:
downloader = WeatherDownloader()
files = downloader.download(station)

print(f"EPW file: {files.epw}")
print(f"DDY file: {files.ddy}")

## Step 4: Explore Design Day Conditions

Parse the DDY file and examine available design days.

In [ ]:
ddm = DesignDayManager(files.ddy)

print(ddm.summary())

## Step 5: Create/Load Your Model

Load an existing model or create one.

In [ ]:
from idfkit import new_document

# Create a simple model (or use load_idf() for an existing file)
model = new_document(version=(24, 1, 0))
model.add("Building", "Chicago Office", north_axis=0, terrain="City")
model.add("Zone", "Zone1", x_origin=0, y_origin=0, z_origin=0)

# ... add materials, constructions, surfaces, etc.

print(f"Model has {len(model)} objects")

## Step 6: Apply Design Days to Model

Inject ASHRAE 90.1 design day conditions.

In [ ]:
added = ddm.apply_to_model(
    model,
    heating="99.6%",  # ASHRAE 90.1 heating
    cooling="1%",  # ASHRAE 90.1 cooling
    include_wet_bulb=True,  # Include wet-bulb design day
    update_location=True,  # Update Site:Location
)

print(f"Added {len(added)} design days:")
for name in added:
    print(f"  {name}")

## Step 7: Verify Model Setup

Check that design days and location were added correctly.

In [ ]:
# Check Site:Location
if "Site:Location" in model:
    location = model["Site:Location"][0]
    print(f"Site Location: {location.name}")
    print(f"  Latitude: {location.latitude}")
    print(f"  Longitude: {location.longitude}")

# Check design days
if "SizingPeriod:DesignDay" in model:
    print(f"\nDesign days in model: {len(model['SizingPeriod:DesignDay'])}")
    for dd in model["SizingPeriod:DesignDay"]:
        print(f"  {dd.name}")

## Step 8: Run Sizing Simulation

Execute a design-day-only simulation for HVAC sizing.

> **Note:** This requires EnergyPlus to be installed.

In [ ]:
# Simulation code (requires EnergyPlus)
"""
from idfkit.simulation import simulate

result = simulate(
    model,
    files.epw,
    design_day=True,  # Design-day-only for sizing
)

print(f"Success: {result.success}")
print(f"Runtime: {result.runtime_seconds:.1f}s")

if result.success:
    # Check sizing results
    tables = result.sql.get_tabular_data(
        report_name="HVACSizingSummary"
    )
    print(f"\nSizing report has {len(tables)} rows")
"""
print("Simulation code shown above (requires EnergyPlus)")

## Summary

This workflow demonstrated:

1. **Geocoding** — Convert address to coordinates with `geocode()`
2. **Station search** — Find nearest station with `index.nearest()`
3. **Download** — Get weather files with `WeatherDownloader`
4. **Design days** — Parse and apply with `DesignDayManager`
5. **Simulation** — Run sizing with `simulate(design_day=True)`

## One-Liner Alternative

For simple cases, use `apply_ashrae_sizing()`:

```python
from idfkit.weather import apply_ashrae_sizing

apply_ashrae_sizing(model, ddy_path=files.ddy, standard="90.1")
```

## Next Steps

- [Parametric Study](parametric-study.ipynb) — Vary parameters and compare
- [Design Days Guide](../weather/design-days.md) — Detailed design day info
- [Station Search Guide](../weather/station-search.md) — Advanced search